In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json


In [2]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='BethHaworth', api_key='tyWi8tBVkGd8Gk4CXbP2')

In [3]:
from wordcloud import WordCloud, STOPWORDS

In [4]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

UsageError: Line magic function `%` not found.


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
len(stopwords)

In [ ]:
stemmer = nltk.stem.PorterStemmer()

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
        print(message)
        print("="*70)

In [5]:
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
lemm = WordNetLemmatizer()
class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(LemmaCountVectorizer, self).build_analyzer()
        return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [6]:
file = open('../data-extract/buspirone.json', encoding="utf8")

tf_vectorizer = LemmaCountVectorizer(max_df=0.95, 
                                     min_df=2,
                                     stop_words='english',
                                     decode_error='ignore')
tf = tf_vectorizer.fit_transform(file)

In [7]:
feature_names = tf_vectorizer.get_feature_names()
count_vec = np.asarray(tf.sum(axis=0)).ravel()
zipped = list(zip(feature_names, count_vec))
x, y = (list(x) for x in zip(*sorted(zipped, key=lambda x: x[1], reverse=True)))

Y = np.concatenate([y[0:15], y[-16:-1]])
X = np.concatenate([x[0:15], x[-16:-1]])

data = [go.Bar(
            x = x[0:50],
            y = y[0:50],
            marker= dict(colorscale='Jet',
                         color = y[0:50]
                        ),
            text='Word counts'
    )]

layout = go.Layout(
    title='Top 50 Words for Buspirone'
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='basic-bar')

/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [9]:
textdata = "../data-extract/buspirone.json"
text = pd.read_json(textdata)

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [10]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bethhaworth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bethhaworth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [13]:
import random
text_data = []
with open('../data-extract/buspirone.json') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['\\"buspirone', 'absolutely', 'nothing', 'generalize', 'anxiety', 'disorder', 'positive', 'negative', 'nothing', 'start', 'week', 'increase', 'still', 'different', 'stop', 'gutted', 'seem', 'safe', 'option', 'antidepressant', 'would', 'recommend.\\', 'take', 'month']
['\\"why', 'everyone', 'extremely', 'dose', 'buspar', 'taking', 'approx', 'years', 'anxiety', 'prescribe', 'every', 'hours', 'total', 'exceed', '150mgs', 'daily', 'buspar', 'hours', 'make', 'absolutely', 'sense', 'someone', 'would', 'taking', 'unless', 'want', 'anxiety', 'symptom', 'hours', 'take', 'years', 'december']


In [17]:
import gensim


In [18]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')


SyntaxError: invalid syntax (<ipython-input-18-fd38a53b19e8>, line 2)

In [1]:

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=9)
for topic in topics:
    print(topic)

NameError: name 'gensim' is not defined

In [5]:
import plotly
plotly.__version__

'3.4.2'

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv('../data-extract/buspirtimeline.csv')

data_table = FF.create_table(df.head())
py.iplot(data_table, filename='buspirtimeline')

In [11]:
trace1 = go.Scatter(
                    x=df['Month'], y=df['Relative Search Frequency'], # Data
                    mode='lines', name='Relative Search Frequency for Buspirone' # Additional options
                   )

layout = go.Layout(title='Buspirone Google Search Frequency (Relative, Global)',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
py.iplot(fig, filename='buspirtimeline')

In [13]:
df = pd.read_csv('../data-extract/loraztimeline.csv')

data_table = FF.create_table(df.head())
py.iplot(data_table, filename='loraztimeline')

In [14]:
trace1 = go.Scatter(
                    x=df['Month'], y=df['Relative Search Frequency'], # Data
                    mode='lines', name='Relative Search Frequency for Lorazepam' # Additional options
                   )

layout = go.Layout(title='Lorazepam Google Search Frequency (Relative, Global)',
                   plot_bgcolor='rgb(230, 230,230)')

fig = go.Figure(data=[trace1], layout=layout)

# Plot data in the notebook
py.iplot(fig, filename='loraztimeline')

In [3]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

import numpy as np
import pandas as pd

df = pd.read_csv('../data-extract/buspirgeo.csv')

data_table = FF.create_table(df.head())
py.iplot(data_table, filename='buspirgeo')

/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [6]:
data = [go.Bar(
            x=df['Relative Frequency of Searches'],
            y=df['Country'],
            orientation = 'h'
)]

py.iplot(data, filename='horizontal-bar')

In [7]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

import numpy as np
import pandas as pd

df = pd.read_csv('../data-extract/lorazgeo.csv')

data_table = FF.create_table(df.head())
py.iplot(data_table, filename='lorazgeo')

In [9]:
data = [go.Bar(
            x=df['Relative Search Frequency'],
            y=df['Country'],
            orientation = 'h'
)]

py.iplot(data, filename='horizontal-bar')